# Get the departments and rooms within each departments

In [1]:
%matplotlib inline
import numpy as np
import compas
from compas.datastructures import Mesh
import compas.datastructures as ds
from compas_plotters import Plotter
import matplotlib.pyplot as plt
import networkx as nx 
import json
from pathlib import Path
from mpl_toolkits.mplot3d import Axes3D

In [2]:
fin_real = open("C:/Users/SID-DRW/OneDrive - Delft University of Technology/Desktop/new_boundaries_0204/GENERATING_GML/output.json")
j_real = json.loads(fin_real.read())

## Get room names and areas

In [3]:
#make dict

def get_names_and_areas_dict(jsonfile):
    cellspaces = []
    room_names = []
    room_areas = []

    for i in jsonfile['PrimalSpaceFeatures']['CellSpace']:
        cellspaces.append(i)
    for cs in cellspaces:
        name = jsonfile['PrimalSpaceFeatures']['CellSpace'][cs]['name']
        room_names.append(name)

    for cs in cellspaces:
        area = jsonfile['PrimalSpaceFeatures']['CellSpace'][cs]['attributes']['area']
        room_areas.append({'area':area})
    
    name_and_area = dict(map(lambda i,j : (i,j) , room_names,room_areas))
    return name_and_area

dict_room_Name_and_area = get_names_and_areas_dict(j_real)
print(dict_room_Name_and_area)
print(len(dict_room_Name_and_area))


{'CENTRALWAITING1AC1': {'area': '127.0993141'}, 'WAITING/ACTIVITYAREA1DC1': {'area': '178.2169708'}, 'ROOF2R02': {'area': '578.7414997'}, 'CORRIDOR1EC1': {'area': '50.950835'}, 'ROOF3R01': {'area': '1729.774807'}, 'MAINMECHANICALROOM2D05': {'area': '226.04322'}, 'CORRIDOR2AC3': {'area': '25.4807999'}, 'CORRIDOR1BC2': {'area': '27.31731525'}, 'TECH.OFFICE2D18': {'area': '82.089156'}, 'CORRIDOR1CC1': {'area': '40.23448'}, 'PHARM.DISP.1A16': {'area': '65.22472'}, 'CORRIDOR1DC3': {'area': '29.65476604'}, 'PHYSICALEXAM1C18': {'area': '55.37747'}, 'LAB1B04': {'area': '51.1950656'}, 'CORRIDOR1BC4': {'area': '41.72148526'}, 'CORRIDOR2BC3': {'area': '24.92304611'}, 'CORRIDOR2BC5': {'area': '20.941674'}, 'RECEIVING/STORAGE1E24': {'area': '115.1199912'}, 'CORRIDOR2AC1': {'area': '39.02683601'}, 'CORRIDOR1CC3': {'area': '10.608154'}, 'PROVIDERCUBICLES1D16': {'area': '65.26142'}, 'CORRIDOR1DC7': {'area': '29.82494475'}, 'CORRIDOR2BC2': {'area': '27.59131613'}, 'BMETWORKSTATION1E16': {'area': '46.73

## Organize rooms to their deparments

In [4]:
def get_department_of_rooms_dict(room_name_area):

    output_data = {}

    # Loop through each item in the list to organize by department
    for key in room_name_area:
        # Extract department character (the last third character of the room name)
        department = 'Department$' + key[-3]
        
        # Check if the department key exists in the dictionary
        if department not in output_data:
            # If not, create a new list for this department
            output_data[department] = []
            
        # Append the room and area tuple to the appropriate department list
        output_data[department].append(key)

    return output_data

new_output_data = get_department_of_rooms_dict(dict_room_Name_and_area)
new_output_data

{'Department$A': ['CENTRALWAITING1AC1',
  'CORRIDOR2AC3',
  'PHARM.DISP.1A16',
  'CORRIDOR2AC1',
  'DENTALWAITING2A11',
  'CORRIDOR1AC5',
  'CORRIDOR1AC3',
  'CORRIDOR2AC4',
  'RMOANALYST1A02',
  'CORRIDOR1AC2',
  'CORRIDOR2AC2',
  'LIBRARY/CONF.ROOM2A01',
  'ADMIN2A14',
  'LOUNGE1A10',
  'DEVELOPING2A13',
  'X-RAY2A12',
  'CORRIDOR1AC4',
  'OHDTR#12A05',
  'TRICAREOFFICE1A07',
  'COUNSELING1A15',
  'M.TOILET1A12',
  'W.TOILET1A09',
  'SUPEROFFICE2A02',
  'GEN.DENT.DTR#52A09',
  'GEN.DENT.DTR#42A08',
  'OHDTR#22A06',
  'PATIENTADMIN.RECEPT.1A01',
  'TRICAREOFFICE1A03',
  'TRICAREOFFICE1A05',
  'TRICAREOFFICE1A06',
  'SEC.OFFICE2A03',
  'OHDTR#32A07',
  'STAFFTOILET1A13',
  'STAIR1AS1',
  'DENTALWAITING2A10',
  'TRICAREOFFICE1A04',
  'PHARM.OFFICE1A08',
  'CORRIDOR2AC5',
  'DENTALSURGEONOFFICE2A04',
  'JAN.1A11',
  'SUPER/NCOIC1A14',
  'STAIR2AS1'],
 'Department$D': ['WAITING/ACTIVITYAREA1DC1',
  'MAINMECHANICALROOM2D05',
  'TECH.OFFICE2D18',
  'CORRIDOR1DC3',
  'PROVIDERCUBICLES1D16',


In [5]:


def find_correct_department(room_name):
    for char in room_name:
        if char.isdigit():
            index = room_name.index(char) + 1
            return 'Department$' + room_name[index].upper()
    return None


def get_depart_room_dict(keys_to_remove, initial_dict):

    for key in keys_to_remove:
        rooms = initial_dict.pop(key, [])  # Remove the entry and get the rooms list
        for room in rooms:
            correct_department = find_correct_department(room)
            if correct_department in initial_dict:
                initial_dict[correct_department].append(room)
            else:
                initial_dict[correct_department] = [room]

    return initial_dict

keys_to_remove = ['Department$1', 'Department$0', 'Department$2', 'Department$5']
corrected_new_department = get_depart_room_dict(keys_to_remove,new_output_data)
corrected_new_department

{'Department$A': ['CENTRALWAITING1AC1',
  'CORRIDOR2AC3',
  'PHARM.DISP.1A16',
  'CORRIDOR2AC1',
  'DENTALWAITING2A11',
  'CORRIDOR1AC5',
  'CORRIDOR1AC3',
  'CORRIDOR2AC4',
  'RMOANALYST1A02',
  'CORRIDOR1AC2',
  'CORRIDOR2AC2',
  'LIBRARY/CONF.ROOM2A01',
  'ADMIN2A14',
  'LOUNGE1A10',
  'DEVELOPING2A13',
  'X-RAY2A12',
  'CORRIDOR1AC4',
  'OHDTR#12A05',
  'TRICAREOFFICE1A07',
  'COUNSELING1A15',
  'M.TOILET1A12',
  'W.TOILET1A09',
  'SUPEROFFICE2A02',
  'GEN.DENT.DTR#52A09',
  'GEN.DENT.DTR#42A08',
  'OHDTR#22A06',
  'PATIENTADMIN.RECEPT.1A01',
  'TRICAREOFFICE1A03',
  'TRICAREOFFICE1A05',
  'TRICAREOFFICE1A06',
  'SEC.OFFICE2A03',
  'OHDTR#32A07',
  'STAFFTOILET1A13',
  'STAIR1AS1',
  'DENTALWAITING2A10',
  'TRICAREOFFICE1A04',
  'PHARM.OFFICE1A08',
  'CORRIDOR2AC5',
  'DENTALSURGEONOFFICE2A04',
  'JAN.1A11',
  'SUPER/NCOIC1A14',
  'STAIR2AS1',
  'OPENTOBELOW2AC1-B',
  'X-RAYALCOVE2A12-A'],
 'Department$D': ['WAITING/ACTIVITYAREA1DC1',
  'MAINMECHANICALROOM2D05',
  'TECH.OFFICE2D18'